In [ ]:
#|hide
from fastrl.test_utils import initialize_notebook
initialize_notebook()

In [ ]:
#|default_exp agents.dqn.target

In [ ]:
#|export
# Python native modules
from copy import deepcopy
from typing import Optional,Callable,Tuple
# Third party libs
import torchdata.datapipes as dp
from torchdata.dataloader2.graph import traverse_dps,DataPipe
import torch
from torch import nn,optim
# Local modulesf
from fastrl.pipes.core import find_dp
from fastrl.memory.experience_replay import ExperienceReplay
from fastrl.loggers.core import BatchCollector,EpochCollector
from fastrl.learner.core import LearnerBase,LearnerHead
from fastrl.loggers.vscode_visualizers import VSCodeDataPipe
from fastrl.agents.dqn.basic import (
    LossCollector,
    RollingTerminatedRewardCollector,
    EpisodeCollector,
    StepBatcher,
    TargetCalc,
    LossCalc,
    ModelLearnCalc,
    DQN,
    DQNAgent
)

In [ ]:
#|hide
import logging
from fastrl.core import default_logging

In [ ]:
#|hide
logging.basicConfig(**default_logging())

# DQN Target
> DQN that uses snapshots from the NN module to stabilize training



## Training DataPipes

In [ ]:
#|export
class TargetModelUpdater(dp.iter.IterDataPipe):
    def __init__(self,source_datapipe=None,target_sync=300):
        self.source_datapipe = source_datapipe
        if source_datapipe is not None:
            self.learner = find_dp(traverse_dps(self),LearnerBase)
            with torch.no_grad():
                self.learner.target_model = deepcopy(self.learner.model)
        self.target_sync = target_sync
        self.n_batch = 0
        
    def reset(self):
        print('resetting')
        self.learner = find_dp(traverse_dps(self),LearnerBase)
        with torch.no_grad():
            self.learner.target_model = deepcopy(self.learner.model)
        
    def __iter__(self):
        if self._snapshot_state.NotStarted: 
            self.reset()
        for batch in self.source_datapipe:
            if self.n_batch%self.target_sync==0:
                with torch.no_grad():
                    self.learner.target_model.load_state_dict(self.learner.model.state_dict())
            self.n_batch+=1
            yield batch

In [ ]:
#|export
class TargetModelQCalc(dp.iter.IterDataPipe):
    def __init__(self,source_datapipe=None):
        self.source_datapipe = source_datapipe
        
    def __iter__(self):
        self.learner = find_dp(traverse_dps(self),LearnerBase)
        for batch in self.source_datapipe:
            self.learner.done_mask = batch.terminated.reshape(-1,)
            with torch.no_grad():
                self.learner.next_q = self.learner.target_model(batch.next_state)
            self.learner.next_q = self.learner.next_q.max(dim=1).values.reshape(-1,1)
            self.learner.next_q[self.learner.done_mask] = 0 
            yield batch

In [ ]:
#|export
def DQNTargetLearner(
    model,
    dls,
    logger_bases:Optional[Callable]=None,
    loss_func=nn.MSELoss(),
    opt=optim.AdamW,
    lr=0.005,
    bs=128,
    max_sz=10000,
    nsteps=1,
    device=None,
    batches=None,
    target_sync=300
) -> LearnerHead:
    learner = LearnerBase(model,dls=dls[0])
    learner = BatchCollector(learner,batches=batches)
    learner = EpochCollector(learner)
    if logger_bases: 
        learner = logger_bases(learner)
        learner = RollingTerminatedRewardCollector(learner)
        learner = EpisodeCollector(learner)
    learner = learner.catch_records()
    learner = ExperienceReplay(learner,bs=bs,max_sz=max_sz)
    learner = StepBatcher(learner,device=device)
    learner = TargetModelQCalc(learner)
    learner = TargetCalc(learner,nsteps=nsteps)
    learner = LossCalc(learner,loss_func=loss_func)
    learner = ModelLearnCalc(learner,opt=opt(model.parameters(),lr=lr))
    learner = TargetModelUpdater(learner,target_sync=target_sync)
    if logger_bases: 
        learner = LossCollector(learner).catch_records()

    if len(dls)==2:
        val_learner = LearnerBase(model,dls[1])
        val_learner = BatchCollector(val_learner,batches=batches)
        val_learner = EpochCollector(val_learner).catch_records(drop=True)
        val_learner = VSCodeDataPipe(val_learner)
        return LearnerHead((learner,val_learner),model)
    else:
        return LearnerHead(learner,model)

Try training with basic defaults...

In [ ]:
from fastrl.loggers.vscode_visualizers import VSCodeDataPipe
from fastrl.envs.gym import GymDataPipe
from fastrl.loggers.core import ProgressBarLogger
from fastrl.dataloading.core import dataloaders

The DQN learns, but I wonder if we can get it to learn faster...

In [ ]:
#|eval:false
# Setup Loggers
def logger_bases(pipe):
    pipe = pipe.dump_records()
    pipe = ProgressBarLogger(pipe)
    return pipe
# Setup up the core NN
torch.manual_seed(0)
model = DQN(4,2)
# Setup the Agent
model.train()
model = model.share_memory()
agent = DQNAgent(model,do_logging=True,min_epsilon=0.02,max_epsilon=1,max_steps=5000)
# Setup the DataBlock
train_pipe = GymDataPipe(
    ['CartPole-v1']*1,
    agent=agent,
    nsteps=2,
    nskips=2,
    firstlast=True,
    bs=1
)
dls = dataloaders(train_pipe)
# Setup the Learner
learner = DQNTargetLearner(
    model,
    dls,
    logger_bases=logger_bases,
    bs=128,
    max_sz=100_000,
    nsteps=2,
    lr=0.01,
    batches=1000,
    target_sync=300
)
learner.fit(7)

In [ ]:
# exp_replay = find_dp(traverse_dps(learner),ExperienceReplay)
# exp_replay.show()

In [ ]:
from fastrl.agents.core import AgentHead,AgentBase
from fastrl.agents.core import SimpleModelRunner,NumpyConverter
from fastrl.agents.discrete import ArgMaxer
from fastrl.memory.memory_visualizer import MemoryBufferViewer

In [ ]:
val_agent = DQNAgent(model,min_epsilon=0,max_epsilon=0)
valid_pipe = GymDataPipe(
    ['CartPole-v1']*1,
    agent=val_agent,
    nsteps=2,
    nskips=2,
    firstlast=True,
    bs=1,
    n=100,
    include_images=False
)
# valid_pipe = VSCodeDataPipe(valid_pipe)
sample_run = [o[0] for o in valid_pipe.catch_records(drop=True)];

In [ ]:
#|hide
#|eval: false
model.eval()

pipe = GymDataPipe(['CartPole-v1']*1,agent=agent,n=1000,seed=0,include_images=True)
pipe = VSCodeDataPipe(pipe)

list(pipe);
pipe.show()
# list(pipe)

In [ ]:
from fastrl.agents.core import StepFieldSelector

In [ ]:
#|export
def DQNValAgent(
    model,
    device='cpu'
)->AgentHead:
    agent_base = AgentBase(model)
    agent = StepFieldSelector(agent_base,field='state')
    agent = SimpleModelRunner(agent).to(device=device)
    # agent = ArgMaxer(agent,only_idx=True)
    # agent = NumpyConverter(agent)
    # agent = PyPrimativeConverter(agent)
    agent = AgentHead(agent)
    return agent

val_agent = DQNValAgent(model)

In [ ]:
MemoryBufferViewer(sample_run,val_agent)

In [ ]:
#|hide
#|eval:false
# learner.validate(epochs=1,batches=200)

In [ ]:
#|hide
#|eval: false
!nbdev_export